<a href="https://colab.research.google.com/github/yuziny/ML_miniproject/blob/main/Attrition_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이직 여부

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

In [28]:
path = 'https://raw.githubusercontent.com/jangrae/csv/master/Attrition_simple2.csv'
data = pd.read_csv(path)

In [29]:
data.head()

,Attrition,Age,DistanceFromHome,EmployeeNumber,Gender,JobSatisfaction,MaritalStatus,MonthlyIncome,OverTime,PercentSalaryHike,TotalWorkingYears
0,0,33,7,817,Male,3,Married,11691,No,11,14
1,0,35,18,1412,Male,4,Single,9362,No,11,10
2,0,42,6,1911,Male,1,Married,13348,No,13,18
3,0,46,2,1204,Female,1,Married,17048,No,23,28
4,1,22,4,593,Male,3,Single,3894,No,16,4


- Attrition: 이직 여부(1: 이직, 0: 잔류)
- Age: 나이
- DistanceFromHome: 집-직장 거리(단위: 마일)
- EmployeeNumber: 사번
- Gender: 성별(Male, Female)
- JobSaticsfaction: 직무 만족도(1: Low, 2: Medium, 3: High, 4: Very High)
- MaritalStatus: 결혼 상태(Single, Married, Divorced)
- MonthlyIncome:월급(단위: 달러)
- OverTime:야근 여부(Yes, No)
- PercentSalaryHike: 전년 대비 급여 인상률(단위: %)
- TotalWorkingYears: 총 경력 연수

In [30]:
data.describe()

,Attrition,Age,DistanceFromHome,EmployeeNumber,JobSatisfaction,MonthlyIncome,PercentSalaryHike,TotalWorkingYears
count,1196.000000,1196.00000,1196.000000,1196.000000,1196.000000,1196.000000,1196.000000,1196.000000
mean,0.163043,36.94398,9.258361,1035.629599,2.716555,6520.104515,15.251672,11.330268
std,0.369560,9.09270,8.166016,604.340130,1.110962,4665.902253,3.625946,7.823821
min,0.000000,18.00000,1.000000,1.000000,1.000000,1009.000000,11.000000,0.000000
25%,0.000000,30.00000,2.000000,507.750000,2.000000,2928.250000,12.000000,6.000000
50%,0.000000,36.00000,7.000000,1028.000000,3.000000,4973.500000,14.000000,10.000000
75%,0.000000,43.00000,14.000000,1581.250000,4.000000,8420.500000,18.000000,15.000000
max,1.000000,60.00000,29.000000,2068.000000,4.000000,19999.000000,25.000000,40.000000


In [31]:
data.isnull().sum()

Attrition            0
Age                  0
DistanceFromHome     0
EmployeeNumber       0
Gender               0
JobSatisfaction      0
MaritalStatus        0
MonthlyIncome        0
OverTime             0
PercentSalaryHike    0
TotalWorkingYears    0
dtype: int64

In [32]:
data.corr()

,Attrition,Age,DistanceFromHome,EmployeeNumber,JobSatisfaction,MonthlyIncome,PercentSalaryHike,TotalWorkingYears
Attrition,1.000000,-0.167866,0.081973,-0.008707,-0.078936,-0.163572,-0.000048,-0.182162
Age,-0.167866,1.000000,-0.010917,-0.023786,-0.012425,0.490107,-0.008303,0.674331
DistanceFromHome,0.081973,-0.010917,1.000000,0.054948,-0.021623,-0.012803,0.052348,0.002606
EmployeeNumber,-0.008707,-0.023786,0.054948,1.000000,-0.022863,-0.014032,-0.009514,-0.016317
JobSatisfaction,-0.078936,-0.012425,-0.021623,-0.022863,1.000000,-0.025082,0.030811,-0.039380
MonthlyIncome,-0.163572,0.490107,-0.012803,-0.014032,-0.025082,1.000000,-0.021334,0.768437
PercentSalaryHike,-0.000048,-0.008303,0.052348,-0.009514,0.030811,-0.021334,1.000000,-0.021988
TotalWorkingYears,-0.182162,0.674331,0.002606,-0.016317,-0.039380,0.768437,-0.021988,1.000000


# 전처리

In [33]:
target = 'Attrition'

x = data.drop(target, axis=1)
y = data.loc[:, target]

In [34]:
dumm_cols = ['Gender', 'JobSatisfaction', 'MaritalStatus', 'OverTime']

x = pd.get_dummies(x, columns=dumm_cols, drop_first=True)
x.head()

,Age,DistanceFromHome,EmployeeNumber,MonthlyIncome,PercentSalaryHike,TotalWorkingYears,Gender_Male,JobSatisfaction_2,JobSatisfaction_3,JobSatisfaction_4,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,33,7,817,11691,11,14,1,0,1,0,1,0,0
1,35,18,1412,9362,11,10,1,0,0,1,0,1,0
2,42,6,1911,13348,13,18,1,0,0,0,1,0,0
3,46,2,1204,17048,23,28,0,0,0,0,1,0,0
4,22,4,593,3894,16,4,1,0,1,0,0,1,0


In [35]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

# 정규화

In [36]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [37]:
x_train[:10]

array([[0.21428571, 0.32142857, 0.67864078, 0.24899726, 0.        ,
        0.15      , 0.        , 0.        , 0.        , 1.        ,
        1.        , 0.        , 1.        ],
       [0.54761905, 0.07142857, 0.98640777, 0.41624446, 0.        ,
        0.35      , 1.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.28571429, 0.07142857, 0.14174757, 0.08475829, 0.        ,
        0.2       , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 1.        ],
       [0.35714286, 0.5       , 0.28009709, 0.66281402, 0.07142857,
        0.375     , 1.        , 0.        , 1.        , 0.        ,
        1.        , 0.        , 1.        ],
       [0.38095238, 0.07142857, 0.53300971, 0.3538632 , 0.42857143,
        0.175     , 1.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        ],
       [0.33333333, 0.42857143, 0.59660194, 0.08929702, 0.64285714,
        0.05      , 0.     

# 모델링

In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

model = KNeighborsClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[280  20]
 [ 45  14]]
              precision    recall  f1-score   support

           0       0.86      0.93      0.90       300
           1       0.41      0.24      0.30        59

    accuracy                           0.82       359
   macro avg       0.64      0.59      0.60       359
weighted avg       0.79      0.82      0.80       359

